## Philosophy

- For every plot record, construct a JSON.
    - Enumerator, Plot, CRops, 

In [1]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from icecream import ic
import json
from datetime import datetime
from copy import deepcopy
from shapely import wkt
load_dotenv()

True

In [2]:
cd ~/Documents/projects/mini_projects/regreen_cleaning/brics3_migration/

/Users/victor/Documents/projects/mini_projects/regreen_cleaning/brics3_migration


In [3]:
data_folder = os.path.join(os.getcwd(),'data')
cleaned_csv = os.path.join(data_folder, 'final_range_data.csv')
dff = pd.read_csv(cleaned_csv)

# #lambda function to filter for bunds
# #for each row, is it a list/str, then is 'bunds' in it, if so, return the df.
cleaned_df = dff[
    dff["land_use_selection-intervention"]
    .apply(lambda x: isinstance(x, (list, str)) and "soil_bunds" in x)
]

# #bunds datasets
# bunds_df = pd.read_csv(os.path.join(data_folder, 'brics_csvs', 'outputs','cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_bunds.csv'))

TEMPLATE_FILE = os.path.join(data_folder,"halfmoon_json_template.json")
with open(TEMPLATE_FILE, "r") as f:
    template = json.load(f)



## enumerators

In [4]:
#sorted(list(dff['basic_info-name'].unique()))
##empty name and key dictionary
names_dict = {}
for idx, row in dff.iterrows():
    #ic({row["KEY"], row["basic_info-name"]})
    names_dict[row["KEY"]] = row["basic_info-name"]
#ic(names_dict)
name_uname_dict = {
    "Zakariye haji ali": "abdullahi.hassan@mardo.org", 
    "Issack Idow Ibrahim": "abdullahi.hassan@mardo.org",
    "Abdullahi Hassan Adan": "abdullahi.hassan@mardo.org",
    "Abdikheir Mohamed": "mukhtar.yusuf23",
    "Abdikheir Mohamed Ali": "mukhtar.yusuf23",
    "Abdikheir Mohamed Dahir": "mukhtar.yusuf23",
    "Ali": "mukhtar.yusuf23",
    "Ali Adan": "mukhtar.yusuf23",
    "Ali Aden": "mukhtar.yusuf23",
    "Fathi": "mukhtar.yusuf23", 
    "Fathi Abdirashid": "mukhtar.yusuf23",
    "Ahmed Omar Hassan": "AhmedOmar",
    "Maria Abdirahman Ali": "AhmedOmar",
    "ABSHIR ABDULLAH ALI": "Ahil",
    "Ahmed Ibrahim": "JubaFoundation",
    "Ahmed Ibrahim Mohamed": "JubaFoundation",
    "Abdiaziz Adan Hassa": "guudow",
    "Abdiaziz Adan Hassan": "guudow",
    "Abdi Hassan Adan": "AbdiHassan",
    "Abdullahi": "sharif1234",
    "Abdirashid Sheikh Mohamed": "NRMO",
    "Abdullahi Sharif": "sharif1234",
    "Abdullahi Sharif LLG": "sharif1234",
    "Abdullahi Sharif Noor LLG": "sharif1234",
    "Abdijalil": "Csxaashi",
    "Abdijalil said": "Csxaashi",
    "Abdikarim Mohamed Ismail": "samandoulgou",
    "Abdikarin Mohamed Ismail": "samandoulgou",
    "Hassan Omar Khadhib": "Khadhib",
    "Hassan Omar Khadhibr": "Khadhib",
    "Jamal Ali": "JamalAli",
    "Jamal Ali Tagal": "JamalAli",
    "Mohamud Abdullahi Ali": "Mohamud",
    "Moulid Abdullahi Abdi": "Moulidwadani",
    "Muktar Mohamednoor Rage": "Mrage1997",
    "Osman Ebey Omar": "iibey",
    "Yusuf Mahi munye": "Mcnbdn617660",
    "abdullahi Sharif LLG": "sharif1234",
    "osman Ebey Omar": "iibey",
}

## match and replace names with usernames
for k, v in names_dict.items():
    for key, value in name_uname_dict.items():
        if v == key:
            names_dict[k] = value
#ic(names_dict)

##enumerator dictionary
enumeratorrr = {
        "id": 3726,
        "first_name": "First",
        "last_name": "Last",
        "gender": "Male",
        "age_category": "18 - 35",
        "phone_number": "0e15704046",
        "email": "test_mail@gmail.com",
        "country": "test",
        "organization": "Test",
        "username": "test",
        "type": "ENUMERATOR"
    }

enums = []
#create enumerator jsons
for key, value in names_dict.items():
    e = deepcopy(enumeratorrr)
    e['username'] = value
    #e['username'] = 'benards'
    e['key'] = key
    # enums.append(
    #     {"enumerator": e, "key": key})
    enums.append(e)
    

#ic(enums)

## Farming Entity

In [5]:
farming_entity_list = []
for idx, i in enumerate(dff.to_dict(orient='records'), start=1):
    farming_entity = {}
    #ic(i['basic_info-organisation_name'])
    farming_entity['id'] = idx
    farming_entity['first_name'] = i['basic_info-organisation_name']
    farming_entity['middle_name'] = i['basic_info-organisation_name']
    farming_entity['last_name'] = i['basic_info-organisation_name']
    farming_entity['organization'] = 'INSTITUTION'
    farming_entity['key'] = i['KEY']
    
    farming_entity_list.append(
        {
            "farmingEntity": farming_entity, 
            'key': i['KEY']
        }
    )
    # ic(farming_entity_list)
    # ic(idx)
    

## project

In [6]:
projects_list = []

names =[]
for row in dff.to_dict(orient='records'):
    name = row['land_use_selection-project_name']
    
    if name == 'terra':
        name = name.upper()
        id = 131
    # elif name == 'RAF':
    #     name = 'Test'
    elif name == 'brcis3':
        name = 'BRCiS III'
        id = 93
    
    names.extend([name])
    
    projects_list.append({
        "project": {
            "id": id,
            "project_name": name
        },
        "key": row['KEY']
    })

ic(set(names))


ic| set(names): {'BRCiS III', 'TERRA', 'RAF'}


{'BRCiS III', 'RAF', 'TERRA'}

In [7]:
halfmoon_df = pd.read_csv(os.path.join(data_folder, 'brics_csvs', 'outputs','cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_halfmoons.csv'))
bunds_df = pd.read_csv(os.path.join(data_folder, 'brics_csvs', 'outputs','cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_bunds.csv'))
contours_df = pd.read_csv(os.path.join(data_folder, 'brics_csvs', 'outputs','cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_contour.csv'))
gabions_df = pd.read_csv(os.path.join(data_folder, 'brics_csvs', 'outputs','cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_gabions.csv'))
swales_df = pd.read_csv(os.path.join(data_folder, 'brics_csvs', 'outputs','cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_swales.csv'))

# dictionary to match intervention to child table
matching_dict = {
    "halfmoons":halfmoon_df,
    "soil_bunds":bunds_df,
    "contour_planting":contours_df,
    "gabions": gabions_df,
    "swales": swales_df
}

# plot-soil_bunds details
- read df to dictionary, for matching keys, extract df values
### plot-points
- parse the geoshape column from cleaned df
- split and extract the values from each i
- append to json keys

## crops
- get matching rows based on key(id)
- for these rows if crop has crops, get its values and 'other' if any
    - every matching rows values are appended to a list(of all crops for that plot)
- clean 'and' from the crop types
 


In [ ]:
import uuid
import copy
import re

plot_list = []
point_id = 1

## helper functions
## match columns via a substring eg crops_in_field with 'managements_contour-crops_in_field5'
def find_column_by_substring(row_dict, substring):
    for k in row_dict.keys():
        if substring in k:
            return k
    return None

## function to extract geoshape str to plot points list
def parse_geoshape(geoshape_str):
    #parse a geoshape string into a list of point dictionaries
    points = []
    parts = geoshape_str.split(';')

    for part in parts:
        part = part.strip()
        if not part:
            continue

        lat, lon, alt, acc = map(float, part.split(' '))

        points.append({
            "longitude": lon,
            "latitude": lat,
            "altitude": alt,
            "accuracy": acc
        })

    return points


parent_records = dff.to_dict(orient='records')
count = 0

#iterate over every i to capture plot details
for idx, i in enumerate(parent_records):
    #if i['KEY'] =='uuid:c9db5e91-ee4d-4098-92ca-1551e55a6c28':
    ic(i['KEY'])
    plot_id = idx + 1
    count += 1

    # ---- BUILD PLOT DETAILS ----
    template_cp = copy.deepcopy(template[0])
    plot_details = template_cp['plot']

    plot_details['id'] = plot_id
    plot_details['key'] = i['KEY']
    plot_details['name'] =  "uuid:" + str(uuid.uuid4())
    plot_details['calculated_size'] = i['area_ha']
    plot_details['photo_url'] = i['plot_details-photo']
    plot_details['farmingEntityId'] = farming_entity['id']
    plot_details['livestock_allowed'] = False if i['land_use_selection-land_use'] == 'agriculture' else True
    plot_details['conservation_area'] = False
    """--------------------------------PLOT POINTS - GEOSHAPE PARSING ----------------------------------"""
    geoshape_str = i['plot_details-polygon']
    
    parsed_points = parse_geoshape(geoshape_str)

    points_list = []
    for p in parsed_points:
        points_list.append({
            "id": point_id,
            "plotId": plot_id,
            "longitude": p["longitude"],
            "latitude": p["latitude"],
            "altitude": p["altitude"],
            "accuracy": p["accuracy"]
        })
        point_id += 1


    """--------------------------------ADMINISTRATIVE DETAILS ----------------------------------"""
    administrative = plot_details['subCounty']
    administrative['subcounty_name'] = i['geography-district_name']
    administrative['county']['county_name'] = i['geography-region_name']


    #update plot_details with number of points and administrative info
    plot_details['points'] = points_list
    plot_details['subCounty'] = administrative
    

    interventions = i['land_use_selection-intervention'].split()

    #all_child_rows for all interventions aggregated for crop extraction  
    all_child_rows =[]

    ic(type(interventions))
    ic(interventions)

    #loop through each intervention df and collect all intervention child rows info for crop extraction.
    for j in interventions:
        if j in matching_dict:
            child_df = matching_dict[j]
            #if no child record for intervention, eg waterpoints, skip crop extraction
            if child_df is None:
                continue


            #get only child row for this key
            matching_child_rows = child_df[child_df['PARENT_KEY'] == i['KEY']]
            if not matching_child_rows.empty:
                all_child_rows.extend(matching_child_rows.to_dict(orient='records'))
            
        else:
            ic('No Child record for', j)
    ic(len(all_child_rows))

    """-------------------------------------CROP DETAILS------------------------------------"""
    all_crops_list = []
    if all_child_rows:
        for x in all_child_rows:
            #ic(x.keys())
            # if pd.isna(x.get('crops_in_field')):
            #     crop_list = []
            # else:
            #     crop_list = str(x['crops_in_field']).split()
            crops_col = find_column_by_substring(x, 'crops_in_field')

            if crops_col is None or pd.isna(x.get(crops_col)):
                crop_list = []
            else:
                crop_list = str(x[crops_col]).split()

            if 'other' in crop_list:
                other_crops_value = re.split(r',|\band\b', str(x.get('other_crops', '')))
                other_crops_value = [c.strip() for c in other_crops_value if c.strip()]

                crop_list = [c for c in crop_list if c != 'other']
                crop_list.extend(other_crops_value)

            all_crops_list.extend(crop_list)

        # cleanup
        all_crops_list = [c for c in set(all_crops_list) if c]
    
        plot_details['crops'] = all_crops_list
        plot_details['has_crops'] = bool(all_crops_list)
    else:
        plot_details['has_crops'] = False
    
    # del plot_details['points']
    # del plot_details['subCounty']
    general_plot = {
        "plot":plot_details, 
        'key':i['KEY']
    }
    #ic(i['KEY'])
    plot_list.append(
        general_plot
    )
ic(len(plot_list))

with open(os.path.join(data_folder, 'to_delete.json'), 'w') as file:
    json.dump(plot_list, file, indent=4)



ic| i['KEY']: 'uuid:f8a33b7a-e769-41f7-b58e-422d1cc80f47'
ic| type(interventions): <class 'list'>
ic| interventions: ['halfmoons', 'swales', 'water_points']
ic| 'No Child record for', j: 'water_points'
ic| len(all_child_rows): 7
ic| i['KEY']: 'uuid:b5ab198a-5c8b-4e40-90c4-f0efa1e969ff'
ic| type(interventions): <class 'list'>
ic| interventions: ['terracing']
ic| 'No Child record for', j: 'terracing'
ic| len(all_child_rows): 0
ic| i['KEY']: 'uuid:93a4de6b-e4d4-42ad-af3d-868aae8bc724'
ic| type(interventions): <class 'list'>
ic| interventions: ['terracing']
ic| 'No Child record for', j: 'terracing'
ic| len(all_child_rows): 0
ic| i['KEY']: 'uuid:feec15d9-9f68-49cd-b251-9816e5cdd302'
ic| type(interventions): <class 'list'>
ic| interventions: ['terracing']
ic| 'No Child record for', j: 'terracing'
ic| len(all_child_rows): 0
ic| i['KEY']: 'uuid:366f2dfd-c43f-4b67-b8a2-b5056ad18b45'
ic| type(interventions): <class 'list'>
ic| interventions: ['halfmoons', 'swales', 'soil_bunds', 'water_points']


## Intervention details

In [9]:
import intervention_specific_logic
import importlib
importlib.reload(intervention_specific_logic)


from intervention_specific_logic import extract_contours_details, extract_halfmoon_details, extract_wp_details, extract_rockdam_details, extract_swales_details, extract_terraces_details, extract_iremoval_details, extract_gabions_details, extract_bunds_details


intervention_mapping = {
    "contour_planting": lambda c, idx: extract_contours_details(c, contours_df, idx),
    "halfmoons": lambda c, idx: extract_halfmoon_details(c, halfmoon_df, idx),
    "water_points": lambda c, idx: extract_wp_details(c, idx),
    "rock_dam": lambda c, idx: extract_rockdam_details(c, idx),
    "swales" : lambda c, idx: extract_swales_details(c, swales_df, idx),
    "terracing": lambda c, idx: extract_terraces_details(c, idx),
    "invasive_removal": lambda c, idx: extract_iremoval_details(c, idx),
    "gabions": lambda c, idx: extract_gabions_details(c, gabions_df, idx),
    "soil_bunds": lambda c, idx: extract_bunds_details(c, bunds_df, idx)
}

all_intervention_results = []
k = 0
#iterate over every c to capture plot details
for idx, c in enumerate(parent_records):
    k += 1
    ic(c['KEY'])
    #get interventions for this record
    interventions = c['land_use_selection-intervention'].split()
    ic(interventions)

    #call only what intervention exists
    intervention_results = {}

    for intervention, extractor in intervention_mapping.items():
        if intervention in interventions:
            ic(f"Extracting {intervention} for {c['KEY']}")
            intervention_results[intervention] = extractor(c, idx)


    #construct json structure for extracted data
    record_dict = {
        k: v
        for intervention_results in intervention_results.values()
        for item in intervention_results      # list of dicts
        for k, v in item.items()               # inner dict
    }

    all_intervention_results.append(record_dict)

    if k == 120:
        break




ic| c['KEY']: 'uuid:f8a33b7a-e769-41f7-b58e-422d1cc80f47'
ic| interventions: ['halfmoons', 'swales', 'water_points']
ic| f"Extracting {intervention} for {c['KEY']}": 'Extracting halfmoons for uuid:f8a33b7a-e769-41f7-b58e-422d1cc80f47'
ic| f"Extracting {intervention} for {c['KEY']}": 'Extracting water_points for uuid:f8a33b7a-e769-41f7-b58e-422d1cc80f47'
ic| f"Extracting {intervention} for {c['KEY']}": 'Extracting swales for uuid:f8a33b7a-e769-41f7-b58e-422d1cc80f47'
ic| c['KEY']: 'uuid:b5ab198a-5c8b-4e40-90c4-f0efa1e969ff'
ic| interventions: ['terracing']
ic| f"Extracting {intervention} for {c['KEY']}": 'Extracting terracing for uuid:b5ab198a-5c8b-4e40-90c4-f0efa1e969ff'
ic| c['KEY']: 'uuid:93a4de6b-e4d4-42ad-af3d-868aae8bc724'
ic| interventions: ['terracing']
ic| f"Extracting {intervention} for {c['KEY']}": 'Extracting terracing for uuid:93a4de6b-e4d4-42ad-af3d-868aae8bc724'
ic| c['KEY']: 'uuid:feec15d9-9f68-49cd-b251-9816e5cdd302'
ic| interventions: ['terracing']
ic| f"Extracting {in

In [10]:

with open(os.path.join(data_folder, 'to_delete.json'), 'w') as file:
    json.dump(all_intervention_results, file, indent=4)



In [11]:
# component lists
lists = [farming_entity_list, all_intervention_results ,plot_list, projects_list]

# The output dictionary
merged = {}

for lst in lists:
    ic(len(lst))
    for item in lst:
        k = item['key']

        #check if already updated
        if k not in merged:
            merged[k] = {}
            
        #merge the item into the existing dictionary
        merged[k].update(item)

final = list(merged.values())

for i in final:
    #match enumerators
    for l in enums:
        if i['key'] == l['key']:
            i['enumerator'] = l


    # i["microcatchment"]= []
    # i["waterpoint"] = []
    # i["iremoval"] = []
    # i["econtrol"] = []
    i["seedbank"] = []
    i["seeding"] = []
    i["boma"] = []
    i["cgrazing"] = []
    i["completed"] = True
    i["rangeModuleId"] = 1
    i["currentStep"]= "DONE"
    i["moduleId"] = 6
    i["is_revisit"] = False

    #get the date_collected
    for j in parent_records:
        if i["key"] == j["KEY"]:
            i["date_collected"] = datetime.strptime(j['basic_info-date'], "%d/%m/%Y").strftime("%Y-%m-%d")
    #match project id
    for x in projects_list:
        if i['key'] == x['key']:
            i['projectId'] = x['project']['id']
    #match plot id
    for k in plot_list:
        if i['key'] == k['key']:
            i['plotId'] = k['plot']['id']
    #match farming_entity_id
    for h in farming_entity_list:
        if i['key'] == h['key']:
            i['farmingEntityId'] = h['farmingEntity']['id']
    #match enumerator id
    for e in enums:
        if i['key'] == e['key']:
            i['enumeratorId'] = e['id']


ic(len(final))


ic| len(lst): 119
ic| len(lst): 119
ic| len(lst): 119
ic| len(lst): 119
ic| len(final): 119


119

In [12]:

with open(os.path.join(data_folder, 'to_delete.json'), 'w') as file:
    json.dump(final, file, indent=4)



- check for required interventions from constructed json.
- add missing keys

In [13]:
required_keys = {
    'econtrol', #soil_bunds, swales, gabions, terracing contour_planting
    'iremoval', 
    'waterpoint',
    'microcatchment'
}

for i in enumerate(final):
    json_file = i[1]    
    for k in required_keys:
        if k not in json_file:
            ic(k, 'not in json file and has been added')
            json_file[k] = []
        else:
            ic(k, 'found')
    ic('-----------------------')        


ic| k: 'econtrol', 'found'
ic| k: 'microcatchment', 'found'
ic| k: 'waterpoint', 'found'
ic| k: 'iremoval', 'not in json file and has been added'
ic| '-----------------------'
ic| k: 'econtrol', 'found'
ic| k: 'microcatchment', 'not in json file and has been added'
ic| k: 'waterpoint', 'not in json file and has been added'
ic| k: 'iremoval', 'not in json file and has been added'
ic| '-----------------------'
ic| k: 'econtrol', 'found'
ic| k: 'microcatchment', 'not in json file and has been added'
ic| k: 'waterpoint', 'not in json file and has been added'
ic| k: 'iremoval', 'not in json file and has been added'
ic| '-----------------------'
ic| k: 'econtrol', 'found'
ic| k: 'microcatchment', 'not in json file and has been added'
ic| k: 'waterpoint', 'not in json file and has been added'
ic| k: 'iremoval', 'not in json file and has been added'
ic| '-----------------------'
ic| k: 'econtrol', 'found'
ic| k: 'microcatchment', 'found'
ic| k: 'waterpoint', 'found'
ic| k: 'iremoval', 'not in 

In [14]:
q = 0
for i in enumerate(final):
    q += 1
    count = len(i[1].keys())
    if count != 23:
        ic('Error!', count, 'not equal to 23 for', i[1]['key'])
    else:
        ic('✅', q, '/', len(final))



ic| '✅', q: 1, '/', len(final): 119
ic| '✅', q: 2, '/', len(final): 119
ic| '✅', q: 3, '/', len(final): 119
ic| '✅', q: 4, '/', len(final): 119
ic| '✅', q: 5, '/', len(final): 119
ic| '✅', q: 6, '/', len(final): 119
ic| '✅', q: 7, '/', len(final): 119
ic| '✅', q: 8, '/', len(final): 119
ic| '✅', q: 9, '/', len(final): 119
ic| '✅', q: 10, '/', len(final): 119
ic| '✅', q: 11, '/', len(final): 119
ic| '✅', q: 12, '/', len(final): 119
ic| '✅', q: 13, '/', len(final): 119
ic| '✅', q: 14, '/', len(final): 119
ic| '✅', q: 15, '/', len(final): 119
ic| '✅', q: 16, '/', len(final): 119
ic| '✅', q: 17, '/', len(final): 119
ic| '✅', q: 18, '/', len(final): 119
ic| '✅', q: 19, '/', len(final): 119
ic| '✅', q: 20, '/', len(final): 119
ic| '✅', q: 21, '/', len(final): 119
ic| '✅', q: 22, '/', len(final): 119
ic| '✅', q: 23, '/', len(final): 119
ic| '✅', q: 24, '/', len(final): 119
ic| '✅', q: 25, '/', len(final): 119
ic| '✅', q: 26, '/', len(final): 119
ic| '✅', q: 27, '/', len(final): 119
ic| '✅', q

In [15]:

with open(os.path.join(data_folder, 'to_delete.json'), 'w') as file:
    json.dump(final, file, indent=4)



## clean up keys

In [17]:
#clean up keys
def remove_keys(obj, key="key"):
    if isinstance(obj, dict):
        obj.pop(key, None)
        for v in obj.values():
            remove_keys(v, key)
    elif isinstance(obj, list):
        for v in obj:
            remove_keys(v, key)
    return obj

#finaal = [x for x in final if x['key'] == 'uuid:b4a2d545-66fe-4d23-825b-41025149b460']
finaal = [remove_keys(i) for i in final]

#add id
for idx, c in enumerate(finaal):
    c['id'] = idx + 1



In [20]:
q = 0
for i in enumerate(finaal):
    q += 1
    count = len(i[1].keys())
    if count != 23:
        ic('Error!', count, 'not equal to 23 for', i[1]['key'])
    else:
        ic('✅', q, '/', len(final))



with open(os.path.join(data_folder, 'brics3_data.json'), 'w') as file:
    json.dump(finaal, file, indent=4)





ic| '✅', q: 1, '/', len(final): 119
ic| '✅', q: 2, '/', len(final): 119
ic| '✅', q: 3, '/', len(final): 119
ic| '✅', q: 4, '/', len(final): 119
ic| '✅', q: 5, '/', len(final): 119
ic| '✅', q: 6, '/', len(final): 119
ic| '✅', q: 7, '/', len(final): 119
ic| '✅', q: 8, '/', len(final): 119
ic| '✅', q: 9, '/', len(final): 119
ic| '✅', q: 10, '/', len(final): 119
ic| '✅', q: 11, '/', len(final): 119
ic| '✅', q: 12, '/', len(final): 119
ic| '✅', q: 13, '/', len(final): 119
ic| '✅', q: 14, '/', len(final): 119
ic| '✅', q: 15, '/', len(final): 119
ic| '✅', q: 16, '/', len(final): 119
ic| '✅', q: 17, '/', len(final): 119
ic| '✅', q: 18, '/', len(final): 119
ic| '✅', q: 19, '/', len(final): 119
ic| '✅', q: 20, '/', len(final): 119
ic| '✅', q: 21, '/', len(final): 119
ic| '✅', q: 22, '/', len(final): 119
ic| '✅', q: 23, '/', len(final): 119
ic| '✅', q: 24, '/', len(final): 119
ic| '✅', q: 25, '/', len(final): 119
ic| '✅', q: 26, '/', len(final): 119
ic| '✅', q: 27, '/', len(final): 119
ic| '✅', q